## Making 10,000 synthetic spectra for detector 'n3' which is type-I format by using a type-II format file from the 'lle' detector.
#### n3:  
n3 is a GBM NaI detector from the Fermi Space Telescope
#### lle:  
LLE is the LAT Low Energy detector from Fermi

In [1]:
from __future__ import division
from xspec import *
#from models import *  # my own models
import os
import numpy
import json
import glob
from collections import OrderedDict

In [2]:
dataDir = "/Users/KimiZ/GRBs2/analysis/LAT/bn080916009/integrated/"
modDir  = "/Users/KimiZ/GRBs2/analysis/LAT/bn080916009/PYXSPEC/GBMwLAT/grbm+powerlaw/"

os.chdir(dataDir)

In [3]:
Xset.chatter = 0

AllData.clear()

### First spectra must be type-II format to trick Fakeit.  This is the one we don't care about.
### 2nd spectra is my type-I format file that I want to make 10,000 synthetic spectra of and store into one type-II file.

In [4]:
# FIRST SPECTRUM MUST BE OF TYPE-II FORMAT.  
s1              = Spectrum('bn080916009_LAT-LLE_srcspectra.pha{1}')
s1              = AllData(1)
s1.response     = 'bn080916009_LAT-LLE_weightedrsp.rsp{1}'
s1.background   = 'bn080916009_LAT-LLE_bkgspectra.bak{1}'
#s1.ignore('*')  
#s1_noticed_chs  = '9, 10, 11, 12, 13, 14, 15, 16'
#s1.notice(s1_noticed_chs)


s2              = Spectrum('glg_tte_n3_bn080916009_v01.pha1')
s2              = AllData(2)
s2.response     = 'glg_cspec_n3_bn080916009_v00.rsp2'
s2.background   = 'glg_tte_n3_bn080916009_v01.bak'
s2.ignore('*')
s2_noticed_chs  = '[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, \
                    21, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, \
                    43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, \
                    58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, \
                    73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, \
                    88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, \
                    102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, \
                    114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125]'
s2.notice(s2_noticed_chs)

#### Setting Model and Parameters

In [5]:
m1 = Model('grbm+powerlaw')

In [6]:
m1.grbm.alpha.values        = [-0.7429, 0.00742, -10.0, -3.0, 2.0, 5.0]
m1.grbm.beta.values         = [-2.177, 0.0217, -10.0, -5.0, 2.0, 10.0]
m1.grbm.tem.values          = [305.505, 3.055, 10.0, 30.0, 6000.0, 7000.0]
m1.grbm.norm.values         = [0.0190,0.00019, 1e-15, 1e-10, 1.0, 1.0]
m1.powerlaw.PhoIndex.values = [2.077, 0.020, -3.0, -2.0, 9.0, 10.0]
m1.powerlaw.norm.values     = [0.0009, 9.925e-06, 1e-15, 1e-10, 1.0, 1.0]

#### Freezing all parameters

In [7]:
# FREEZE ALL PARAMETERS
for i in range(1, AllModels(1).nParameters+1):
    Parameter.frozen.fset(AllModels(1)(i), 1)

#### Settings

In [8]:
# SOME OF THESE MAY NOT BE NECESSARY FOR FAKEIT
Plot.xAxis              = 'keV'
Fit.method              = 'leven'
Fit.statMethod          = 'cstat'
Fit.nIterations         = 1000
Xset.parallel.leven     = 4

Fit.statMethod          = 'pgstat 1'
Fit.statMethod          = 'cstat 2'

### This is the file I don't care about.  It's used to trick Fakeit:

AllData(1).fileName, 

AllData(1).response.rmf, 

AllData(1).background.fileName,

AllData(1).exposure

    ('bn080916009_LAT-LLE_srcspectra.pha',
     'bn080916009_LAT-LLE_weightedrsp.rsp{1}',
     'bn080916009_LAT-LLE_bkgspectra.bak',
     58.236462396441226)

### This is the file I want to run Fakeit on:  (it's type-I)

AllData(2).fileName, 

AllData(2).response.rmf, 

AllData(2).background.fileName,

AllData(2).exposure

    ('glg_tte_n3_bn080916009_v01.pha1',
     'glg_cspec_n3_bn080916009_v00.rsp2',
     'glg_tte_n3_bn080916009_v01.bak',
     62.666866302490234)

# Fakeit Settings

#### The Output filename is the ONLY one with a '+' sign in the name.
If left with a '+' sign in the name, it will output to a type-I file.

As soon as that's removed and replaced with a '-' sign, it works again and outputs to type-II format.

In [9]:
#  First one is for the detector I wish to run fakeit for 
#  and make 10,000 synthetic spectra.  
#  It is a PHA type-I file.  
FS_det = FakeitSettings(response = 'glg_cspec_n3_bn080916009_v00.rsp2', 
                           background ='glg_tte_n3_bn080916009_v01.bak', 
                           exposure = AllData(2).exposure, 
                           fileName = "fakespec_n3_grbm+pow_-01-_L_.fak")


#  This one is a PHA-II file just to trick Fakit into outputting type-II format.
FS_lle = FakeitSettings(response = 'bn080916009_LAT-LLE_weightedrsp.rsp{1}', 
                        background = 'bn080916009_LAT-LLE_bkgspectra.bak{1}', 
                        exposure = AllData(1).exposure)

Making an array of 10,000 of the fakeit settings for my file.  Then inserting the settings for the file I don't care about at the beginning of the list.

In [10]:
# MAKE A LIST OF THE SAME FAKEIT SETTINGS 10,000 TIMES.

nSpec = 10001        # 1 + number of spectra you want to create.
settingsFake = nSpec * [FS_det] # 1 + nSpectra you want to create
settingsFake.insert(0, FS_lle)

print len(settingsFake)

10002


This is just to show that the first one is the trick file, the rest are the settings I want for the good file.

In [11]:
settingsFake[0:5]

### Sending it off!

In [12]:
AllData.fakeit(len(settingsFake), settingsFake)